In [29]:
import pandas, numpy as np,re
import xml, itertools, re, xml.etree.ElementTree as ET
from html.parser import HTMLParser
import datetime, tqdm
pars =HTMLParser()
def findInXML(s, et): 
    if isinstance(et, str):
        raise ValueError('et should be a tree, not a string')
    r =et.find('.//row[@NombreCampo="%s"]' % s)
    if r is not None:
        return r.get('ValorCampo')
    return None
def prettyPrintXML(s):
    r = xml.dom.minidom.parseString(s) #r.RegistroXML)
    print(pars.unescape(r.toprettyxml()))
    
#from datetime import datetime as dt
#import locale
#locale.setlocale(locale.LC_ALL, 'es_es')
#spanishDateToPandas = lambda s: pandas.to_datetime(dt.strptime(s, '%d %b %Y %H:%M:%S'))

In [2]:
import re
# Maybe I should start tokenizing
cleanWhites = re.compile("[^\S\n]+")
import unicodedata
def remove_diacritics(text):
    """
    Returns a string with all diacritics (aka non-spacing marks) removed.
    For example "Héllô" will become "Hello".
    Useful for comparing strings in an accent-insensitive fashion.
    """
    normalized = unicodedata.normalize("NFKD", text)
    return "".join(c for c in normalized if unicodedata.category(c) != "Mn")

def cleanString(text, removeChars = '-:,;', removeWords = []):
    for c in removeChars:
        text = re.sub('(?<![0-9])\%s' %c, ' ',  text)
        text = re.sub('\%s(?![0-9])' %c, ' ',  text)

    text = cleanWhites.sub(' ', text)
    for w in removeWords:
        text = text.replace(' ' + w + ' ', ' ')
    return text.strip()

# Data reading and basic cleaning

- Find candidates of repeated pateints
- Find the used codes for diagnosis and procedures

In [3]:
casos = pandas.read_csv('Venezolanas/casos.csv', index_col = 0)
pacientes = pandas.read_csv('Venezolanas/pacientes.csv', index_col = 0)
registros = pandas.read_csv('Venezolanas/registros.csv', index_col = 0)
diagnosis = pandas.read_csv('Venezolanas/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv('Venezolanas/procedimientos.csv', index_col = 0)
procedimientosDesc = pandas.read_csv('Venezolanas/procedimientosID.csv', index_col = 0)

#str2Date = lambda s:datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S.%f') if isinstance(s, str) \
#           else datetime.datetime(1, 1, 1)
registros.FechaInicioAtencion = registros.FechaInicioAtencion.map(pandas.to_datetime) #registros.FechaInicioAtencion.map(str2Date)
procedimientos.FechaDescripcion  = procedimientos.FechaDescripcion.map(pandas.to_datetime)
from html.parser import HTMLParser
pars =HTMLParser()
#pars.unescape
registersByCase = registros.groupby('Caso')


C:\Users\CMRC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,27,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\CMRC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
procedureIDFound = set()
abortions = []
def getProcFromXMLString(s):
    s = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', s)
    return s[0]

for k, p in procedimientos.iterrows():
    et = xml.etree.ElementTree.fromstring(p.XmlDescripcion)
    s = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', p.XmlDescripcion)
    for ss in s:
        procedureIDFound.add(ss)
    if 'H3094' in p.XmlDescripcion:
        abortions.append(k)
procedureIDFound = list(sorted(procedureIDFound))

In [5]:
procedimientosDesc.loc[procedureIDFound].to_excel('procedimientosFiltered.xlsx')
procedimientosDesc.loc[procedureIDFound].to_csv('procedimientosFiltered.csv')

In [6]:
unique = registros.Diagnostico.unique()
unique = [u for u in unique if isinstance(u, str)]
for u in unique:
    if u.upper() not in diagnosis.index:
        print(u)

0800


In [7]:
#Check duplicates by birthData
def candidateToRepeated(p1, p2):
    """
    Selects as candidate if they were born in the same name:
    - they have the same 1st surname
    - [TODO] they have similar height (~2cm), as found in the cases
    
    TODO: beware the case of twins!
    """
    #First surname criterio
    if p1[1].Ape1Afil.strip().lower() == p2[1].Ape1Afil.strip().lower():
        return True
    #TODO: Search height and see it's similar
    if False:
        return True
    #Otherwise return False
    return False
    

#Beware
candidates = []
threshold = '2010'
for c in pacientes.groupby('FechaNac'):
    #Arbitrary threshold for separating between mothers and borns
    if len(c[1]) >= 2 and c[0][:4] < threshold:
        for p1, p2 in itertools.combinations(c[1].iterrows(), r = 2):
            if p1[0] != p2[0] and candidateToRepeated(p1, p2):
                candidates.append([p1[0], p2[0]])
pandas.DataFrame.from_records(candidates).to_csv('repeatedCandidates.csv')

# Select the cases by procedure

- Based on the procedure, select the history register associated.
- TODO: Check that diagnosis and procedure are compatible

In [8]:
registersByPatient = registros.groupby('NumeroHistoria')

In [9]:
interventionToCase = {}
caseToProcedure = {}
for i,r  in tqdm.tqdm_notebook(registros.iterrows()):
    if isinstance(r.RegistroXML, str) and '<row NombreCampo="IdDescripcion"' in r.RegistroXML:
        et = ET.fromstring(r.RegistroXML)
        idDescripcion = int(et.find('.//row[@NombreCampo="IdDescripcion"]').get('ValorCampo'))
        interventionToCase[idDescripcion] = (r.Caso, r.NumeroHistoria, i)
        caseToProcedure[idDescripcion] = getProcFromXMLString(procedimientos.loc[idDescripcion].XmlDescripcion)

In [10]:
errors = []
for i, (_, k, k2) in interventionToCase.items():
    if procedimientos.loc[i].IdPaciente != k:
        print('ERROR: Patient name does not coincides', i, k2)
        errors.append(i)
for i in errors:
    del interventionToCase[i]

ERROR: Patient name does not coincides 61066 1356698


# Parse information from cases
Now we need to parse the information from the cases

In [11]:
def getStateDataFromCase(case):
    """
    Get some information from the case.
    """
    data = {}
    r = ET.fromstring(case.RegistroXML)
    try:
        data['FC']=r.find('.//row[@NombreCampo="FrecuenciaCardiaca"]').get('ValorCampo')
        data['FR']=r.find('.//row[@NombreCampo="FrecuenciaRespiratoria"]').get('ValorCampo')
        data['Presion']=r.find('.//row[@NombreCampo="Presion"]').get('ValorCampo')
        data['Temperatura']=r.find('.//row[@NombreCampo="Temperatura"]').get('ValorCampo')
        data['Talla']=r.find('.//row[@NombreCampo="Talla"]').get('ValorCampo')
        data['Peso']=r.find('.//row[@NombreCampo="Peso"]').get('ValorCampo')
        return data
    except:
        return {}

In [12]:
floatParse = '[0-9]*[\.,]?[0-9]+'
compareGPCA = lambda s1, s2: all([i == j or not i or not j for i, j in zip(s1, s2)])
allEqual = lambda s: all([compareGPCA(i, j) for i, j in  itertools.combinations(s, r = 2)])
sep= '\s*[,;:]?\s*'
 #In some cases there is only G
patternGPAC = 'G%s(?P<G>[0-9]+)' %sep + sep + 'P%s(?P<P>[0-9]+)'  %sep + sep + '(?:'
patternGPAC += 'A%s(?P<A>[0-9]+)' %sep  + sep + 'C%s(?P<C>[0-9]+)'  %sep
patternGPAC += '|'
patternGPAC += 'C%s(?P<C2>[0-9]+)' %sep  + sep + 'A%s(?P<A2>[0-9]+)' %sep
patternGPAC += '|' # C and A can be missing, or in other order
patternGPAC += ')'
searchGPAC = re.compile(patternGPAC, flags=0)

meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
meses = meses + list(map(lambda s: s[:3], meses))
noRecuerda = ['NO', '\?']
separadorFecha = '(?:[\.\\/-]|DE|DEL|\s)'
date =  '\(?' +  '((?:[0-9]+)'+ sep + separadorFecha + sep + '(?:[0-9]+|%s)'%  '|'.join(meses) + \
                       sep + separadorFecha + sep + '(?:[0-9]+))' + '\)?' 

searchFUM = re.compile('FUM'+ sep + '(?::|.)?'+ sep +'(' +  date + '|%s)' % ('|'.join(noRecuerda)), flags = re.IGNORECASE)
errors = []
def parseGPCA_and_fum(s):
    """
    Gets the GPCA and FUM from the Triage or epicrisis.
    
    NOTE: some of the cases are incorrect, double check
    TODO: actually, registro del recien nacido has it as a field.
    """
    r = list(filter(lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',ET.fromstring(s).findall('row')))
    if len(r) == 0:
         return {}
    text = pars.unescape(r[0].attrib['ValorCampo'])
    
    allowedStarts = ['7-&gt;', '-', '- ANTECEDENTES']
    line = re.findall('^%s(?:%s)?%s' % (sep, '|'.join(allowedStarts), sep) + 'G' + sep + '[0-9]+.*$', text, re.M)
    if len(line) != 1:
        #If it doesn't find it, search brutally
        search = searchGPAC.findall(s)
        searchCleaned = [(s[0], s[1], s[2] + s[-1], s[3] + s[-2]) for s in search]
        if search and allEqual(search):
            GPCA_OK = True
            parsedGPCA = search[0]
        else:
            GPCA_OK = False
            parsedGPCA = []

    else:
        f = lambda s: s[0] if s else ''
        G = f(re.findall('G' + sep + '([0-9]+)', line[0]))
        A = f(re.findall('A' + sep + '([0-9]+)', line[0]))
        C = f(re.findall('C' + sep + '([0-9]+)', line[0]))
        P = f(re.findall('P' + sep + '([0-9]+)', line[0])) 
        parsedGPCA = [G, P, A, C]
        GPCA_OK = True
    #Prob athere is a better way...
    parsedFUM = searchFUM.findall(s)
    return {'fum' : len(parsedFUM) > 0,
            'fum_OK' : len(parsedFUM) ,
            'GPCA_OK' : GPCA_OK,
            'fum_Data' : parsedFUM,
            'parsedGPCA' : parsedGPCA}

In [13]:
from IPython.display import display

def getBackgroundRegister(casoID, registrosByCase):
    """
    Gets a register that includes the personal background in order Epicrisis / Triage / Ingreso a urgencias
    """
    #
    epicrisis = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Epicrisis', case = False) == True)]
    if epicrisis is not None and len(epicrisis):
        return epicrisis, 'epi'
    
    triage = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Triage', case = False) == True)]
    if triage is not None and len(triage):
        return triage, 'tri'

    urgencias = registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Ingreso de Urgencias', case = False) == True)]
    if urgencias is not None:
        return urgencias, 'ur'


In [14]:
errorsBackgroundRegister = []
backgroundByCase = {}
dataGPAC_fum = {}
typeCases = {}
for k, (case, pId, _) in tqdm.tqdm_notebook(interventionToCase.items()):
    if case is False:
        continue
        
    backgroundByCase[case], typeCases[case] = getBackgroundRegister(case, registersByPatient.get_group(pId))
    if len(backgroundByCase[case]) == 0:
        errorsBackgroundRegister.append(case)
    else: 
        dataGPAC_fum[case] = parseGPCA_and_fum(backgroundByCase[case].iloc[0].RegistroXML)
        c = getProcFromXMLString(procedimientos.loc[k].XmlDescripcion)
        #dataGPAC_fum[case] ['ProcCode'] = c
        #dataGPAC_fum[case] ['ProcDesc'] = procedimientosDesc.loc[c].DESCRIP
    
print('No background', len(errorsBackgroundRegister))

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.



No background 5


In [15]:
def getAllProceduresAndMedicamentsFromCase(registros, Caso):
    procedimientos = []
    medicamentos = []
    for r,v in registros[registros.Caso == Caso].iterrows():
        et = ET.fromstring(v.RegistroXML)
        p = findInXML('ProcedimientosAsociados', et)
        m = findInXML('MedicamentosAdministrado', et)
        if p:
            procedimientos += [p]
        if m:
            medicamentos += [m]
    return procedimientos, medicamentos

medicationByCase = {}
proceduresByCase = {}
for k, (case, pId, _) in tqdm.tqdm_notebook(interventionToCase.items()):
    proceduresByCase[case], medicationByCase[case] = getAllProceduresAndMedicamentsFromCase(registros, case)

In [16]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [17]:
patientsWithPeniciline = []
for c, v in medicationByCase.items():
    if 'MD0441' in ''.join(v):
        patientsWithPeniciline.append(c)
        print(''.join(v))
        


Fecha: 05/02/2017
-19972965-01 - SOLUCION SALINA 0.9 %- Cantidad: 1
-201128 - Jeringa Desechable 1 ml x 27 G/ 1/2&amp;quot; - Cantidad: 1
-IM5064 - LACTATO DE RINGER - Cantidad: 2
-INS1021 - CATGUT 2-0 CT-1 (HR37S) x 90 CM - Cantidad: 1
-INS121 - SONDA NELATON # 8 - Cantidad: 1
-INS224 - Jeringa Desechable 5 ml x 21 G/1 1/2&amp;quot; - Cantidad: 1
-INS225 - Jeringa Desechable 10 ml x 21 G/1 1/2&amp;quot; - Cantidad: 1
-MD0006 - ACETAMINOFEN - Cantidad: 20
-MD0098 - CEFRADINA  - Cantidad: 28
-MD0239 - FITOMENADIONA - Cantidad: 1
-MD0259 - GENTAMICINA 3 % - Cantidad: 1
-MD0430 - OXITOCINA - Cantidad: 2
Fecha: 06/02/2017
-IM5041 - PENICILINA G SODICA (100000 UI)- Cantidad: 1
-IM5064 - LACTATO DE RINGER - Cantidad: 4
-MD0006 - ACETAMINOFEN - Cantidad: 4
-MD0098 - CEFRADINA  - Cantidad: 4
-MD0441 - PENICILINA G BENZATINICA - Cantidad: 1
-MD0445 - PENICILINA G SODICA O POTASICA CRISTALINA - Cantidad: 1
Fecha: 07/02/2017
-MD0006 - ACETAMINOFEN- Cantidad: 28
-MD0098 - CEFRADINA  - Cantidad: 2

In [18]:
len(patientsWithPeniciline)

86

In [19]:
testSifilis = '906039 - TREPONEMA PALLIDUM, ANTICUERPOS PARA SIFILIS, PRUEBAS RAPIDAS  POR INMUNOCROMATOGRAFIA'
vdrl = '906916' #Prueba no treponemica a sifilis
vih1 = '906249'
vih2 = '906249'

In [20]:
patientsTestedForSifilis = []
patientsTestedForVIH = []
for c, v in proceduresByCase.items():
    v = ''.join(v)
    if vih1 in v:
        patientsTestedForVIH.append(c)  
    if testSifilis in v or vdrl in v:
        patientsTestedForSifilis.append(c)  

In [21]:
len(patientsTestedForSifilis)

2210

In [22]:
casesIdx = list(proceduresByCase.keys())
def printMedication(c):
    print(''.join(medicationByCase[c]))
interact(lambda i: printMedication(casesIdx[i]), i = (0, len(casesIdx)))

interactive(children=(IntSlider(value=1232, description='i', max=2464), Output()), _dom_classes=('widget-inter…

<function __main__.<lambda>(i)>

In [23]:
for c, v in casos.iterrows():
    if v.Reingreso:
        print(c, v.Reingreso, v.EstadoSalida)

AD362553 True Vivo


In [24]:
for c, v in proceduresByCase.items():
    if c not in casos.index:
        print(c)

NN


# Check for sifilis in diagnosis

In [ ]:
for i, v in backgroundByCase.items():
    #prettyPrintXML(v.iloc[0].RegistroXML)
    break

## Quality check
- See how many have epicrisis

In [ ]:
findInXML = lambda  s, et: et.find('.//row[@NombreCampo="%s"]' % s).get('ValorCampo')

def ProcessRecienNacido(text):
    et = ET.fromstring(text)
    findInXML = lambda  s: et.find('.//row[@NombreCampo="%s"]' % s).get('ValorCampo')
    fecha = findInXML('InputText_FechaHoraNacimiento')
    hora = findInXML('ASPxTimeEdit_HoraNacimiento')
    EG = findInXML('EdadGestacapuroo')
    EG2 = findInXML('InputText_EdadGestacDubowitzModificado')
    partoVag = findInXML('TexTarea_PartoVaginal')
    partoC = findInXML('TexTarea_PartoCesaria')
    APGAR = findInXML('InputText_APGAR')
    sexo = findInXML('ASPxComboBox_Sexo')
    vivo = findInXML('InputRadio_VM')
    #Antrhopometrics
    weight = findInXML('InputText_Peso')
    height = findInXML('InputText_Talla')
    cc = findInXML('InputText_CC')
    
    #As a check GAPC
    G = findInXML('InputText_ObtetricosGestaciones')
    A = findInXML('InputText_ObtetricosAbortos')
    P = findInXML('InputText_ObtetricosPartos')
    C = findInXML('InputText_ObtetricosCesarias')
    
    #If exposed to sifilis/VIH, it should be here 
    

In [ ]:
len(registros[registros.Asunto == 'Registro del recién nacido'].Caso.unique())

In [ ]:
vivos, muertos = 0, 0

for k, p in registros[registros.Asunto == 'Registro del recién nacido'].iterrows():
    k
    et = ET.fromstring(p.RegistroXML)
    if findInXML('InputRadio_VM', et) == 'Muerto':
        muertos += 1
    else:
        vivos += 1
    if 'desgarros' in p.RegistroXML.lower():
        print('found')
        break
print(vivos, muertos, muertos/(vivos + muertos))

In [ ]:
import re
pos = []
for p, k in backgroundByCase.items():
    try:
        xmlET = ET.fromstring(k.iloc[0].RegistroXML)
        evoluciones = cleanString(xmlET.find('.//row[@NombreCampo="Evoluciones"]').get('ValorCampo').upper())

        if re.search('VDRL (REACT|POS)', evoluciones):
            pos.append(k)
    except:
        print('No epicrisis', p)

In [ ]:
for n in pos:
    if n.iloc[0].Caso not in patientsWithPeniciline:
        print(n.iloc[0].Caso)
        if n.iloc[0].Caso == 'AD362770':
            break

## Some of the ones with sifilis reactive are without penilicine

Probably some of the ones with previous treated sifilis

In [ ]:
alta = 0
noAlta = 0
for k, b in backgroundByCase.items():
    if len(b) == 0:
        continue
    if all(b.AltaMedica):
        alta += 1
    else: 
        noAlta += 1
print(alta, noAlta)

In [ ]:
b.columns

In [ ]:
def caseToBackground(case, backgroundByCase):
    p = backgroundByCase[case]
    if len(p) != 0:  
        text = p.iloc[0].RegistroXML
    #prettyPrintXML(text)
    r = list(filter(
        lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',
        ET.fromstring(text).findall('row')))
    text = remove_diacritics(pars.unescape(r[0].attrib['ValorCampo']))
    return text

In [ ]:
case, _, _ = list(interventionToCase.values())[423]
print(case)
p = backgroundByCase[case]
if len(p) != 0:  
    text = p.iloc[0].RegistroXML
    #prettyPrintXML(text)
    r = list(filter(
        lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',
        ET.fromstring(text).findall('row')))
    textClean = remove_diacritics(pars.unescape(r[0].attrib['ValorCampo']))
    print(textClean)
else:
    print('Error')

# Background

Check from the yes / no form. 

The ones that have, try to get the most common. The rest note for manual

In [31]:
registersByCase
for i, (case, _, _) in interventionToCase.items():
    r = registersByCase.get_group(case)
    r = r[r.Asunto.str.contains('Ingreso de Urgencias') == True]
    if len(r) == 0:
        continue
        
    et = ET.fromstring(r.iloc[0].RegistroXML)
    findInXML('aFarmacologicos', et) == "true"
    findInXML('aGinecoObstetrico', et)  == "true"
    findInXML('aHospitalarios', et)  == "true"
    findInXML('aTraumaticos', et)  == "true"
    if findInXML('aPatologicos', et)  == "true":
        break
    findInXML('aQuirurgicos', et)  == "true"
    findInXML('aToxico', et)  == "true"
    findInXML('aTranfusionales', et)  == "true"
    


In [ ]:
findInXMLnd('aFarmacologicos', et) == "true"
findInXMLnd('aGinecoObstetrico', et)  == "true"
findInXMLnd('aHospitalarios', et)  == "true"
findInXMLnd('aTraumaticos', et)  == "true"
findInXMLnd('aPatologicos', et)  == "true"
findInXMLnd('aQuirurgicos', et)  == "true"
findInXMLnd('aToxico', et)  == "true"
findInXMLnd('aTranfusionales', et)  == "true"

In [32]:
prettyPrintXML(registros[(registros.Caso == case) & registros.Asunto.str.contains('Ingreso de Urgencias')].iloc[0].RegistroXML)

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  from ipykernel import kernelapp as app


<?xml version="1.0" ?>
<C Asunto="Ingreso de Urgencias" Caso="AD209828" CentroA="01" CodigoDiagnosticoRelacionado1="" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="O200" IdPaciente="161838" Modulo="03" Padre="NULL" PlanAdm="100" Prestador="8865876" Raiz="NULL" Registro="07">
	<row NombreCampo="TipoConsulta" NombreTabla="CamposTexto" ValorCampo="OBSTETRICA"/>
	<row NombreCampo="PlanAdministradora" NombreTabla="CamposTexto" ValorCampo="DEPARTAMENTO ADMINISTRATIVO DISTRITAL DE SALUD (DADIS) / DADIS CARTAGENA SISBENIZADO "/>
	<row NombreCampo="MotivoConsulta" NombreTabla="CamposTexto" ValorCampo="&quot;ESTOY EMBARAZADA Y ESTOY SANGRANDO&quot;
"/>
	<row NombreCampo="EnfermedadActual" NombreTabla="CamposTexto" ValorCampo="PACIENTE FEMENINA CON CUADRO CLINICO DE 2 HORAS DE EVOLUCION CONSITENTE EN SANGRADO ESCASO POR GENITALES ASOCIADO A DOLORES TIPO COLICO EN HEMIABDOMEN INFERIOR. POR LO QUE CONSULTA "/>
	<row NombreCampo="FrecuenciaCardiaca" NombreTabla="Camp

# Paraclinics and background familiar

- From the antecedentes? There are many without anything in the venezuelans. Are hey applied during labour?
- Much variability -> need to clean manually, but it might be too costly

See what are the tests done (in orders, and the medication)

In [ ]:
#Parse paraclinics
#VIH
#VDRL -
# PRUEBA RAPIDA DE SIFILIS
dict = {}
for (case, _, _) in interventionToCase.values():
    p = backgroundByCase[case]
    if len(p) == 0:
        continue
    text = p.iloc[0].RegistroXML
    #prettyPrintXML(text)
    r = list(filter(
        lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML" or S.attrib['NombreCampo'] == 'AntecedentesPersonales',
        ET.fromstring(text).findall('row')))
    if len(r) == 0:
        continue
       # text = remove_diacritics(pars.unescape(r[0].attrib['ValorCampo']))
    dict[case] = {'VIH': 'VIH' in text, 'VDRL' : 'VDRL' in text, 'PARACLINICOS' :  'PARACLINICOS' in text, 'SIF' : 'PRUEBA RAPIDA DE SIFILIS' in text}

In [ ]:
paraclinics = pandas.DataFrame.from_dict(dict, orient = 'index')
paraclinics.sum(), len(paraclinics)

In [ ]:
def parseAntecedentes(t):
    t = t.upper()
    antecedentes = []
    negative = ['NO', 'NIEG', 'SIN DATOS', 'NEGATIVO', 'NO REFIERE', 'SIN']
    if 'HIPERTEN' in t or 'HTA' in t:
        antecedentes.append('HTA')
    if any([n in t for n in negative]):
        antecedentes.append('None')
    if 'DIAB' in t or 'DM' in t:
        antecedentes.append('Diabetes')
    if 'ASMA' in t:
        antecedentes.append('Asma')
    if 'CARDIO' in t:
        antecedentes.append('Cardo')
    if 'PREECLAMPSIA' in t:
        antecedentes.append('Preclampsia')

    return antecedentes

In [ ]:
j  = 0
noFam, noPara = 0, 0
fam = {}
para = {}
for k, r in paraclinics.iterrows(): 
    text = caseToBackground(k, backgroundByCase)
    paraclinicos = re.findall('PARACLINICOS(.*)FAMILIARES', text, re.DOTALL)
    familiares = re.findall('FAMILIARES(.*)ALERGICOS', text, re.DOTALL)
    if not paraclinicos:
        noPara += 1
    else: 
        
    else:
        para[k] = cleanString(paraclinicos[0])
    if not familiares:
        noFam += 1
    else:
        print(cleanString(familiares[0]))
        fam[k] = parseAntecedentes(cleanString(familiares[0]))
print(noFam, noPara, len(paraclinics))

In [ ]:
for c in any.values():
    print(c)

# Echographies:

- There is a lot of variability that makes it difficult to get all of them.
- In most cases I should be able to get at least one.
- Do-able: GA. If there is something more, do it manually

In [ ]:
noEchos = ['no trae ecografia', 'ni ecografia']

In [ ]:

embarazo = ['embarazo de', 'emb de', 'embarazo', 'emb', 'QUE REPORTA EMBARAZO DE']
query = 'Ecografi[a]?[s]?' + sep +  '(' + sep +  date + sep + '(?:%s)' % '|'.join(embarazo) + blanks + '(%s)' % floatParse  + blanks + semanas \
+ blanks  + '(:?'+ paraHoy + floatParse +  ')?' + ')+'
re.findall(query, text, re.IGNORECASE)

In [ ]:
blanks = '\s*'
paraHoyPossibilities = ['Para hoy', 'hoy', 'a hoy', 'para el dia de hoy']
paraHoy = '(?:%s)' % '|'.join(paraHoyPossibilities)
semanas = '(?:%s)' % '|'.join(['semanas', 'sem', 'sems'])

echoLine =  sep +  date +  sep + '(?:%s)' % '|'.join(embarazo) + blanks + '(' + '(%s)' % floatParse  + blanks + semanas  + '[,]?'\
+ blanks + paraHoy + blanks + floatParse + blanks + '(:?%s)?' % semanas + ')?'

queryEchos = '(Eco[g]?[r]?[a]?[f]?[i]?[a]?[s]?' + sep +  '(' + echoLine  + ')+)'
def searchEchos(text):
    for t in re.findall(query, text, re.IGNORECASE):
        print(t)
    #TODO: afterwards parse all echographies from the result

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


In [ ]:
backgroundByCase['AD301211'] = getBackgroundRegister(case, registersByPatient.get_group(pId))

In [ ]:
registros[registros.NumeroHistoria == 216835]

In [ ]:
#TODO: check that operation and diagnosis are compatible.

In [ ]:
getprocID(i.XmlDescripcion)
for k,c in interventionToCase[k].iterrows():
    prettyPrintXML(c.RegistroXML)

# Patient information

In [ ]:
def getDataFromPatient(p):
    """
    Get the patient data
    """
    r = {}
    r['id'] = r.Identificacion
    r['marital'] = p.EstadoCivil
    r['etnia'] = p.Etnia #Map from id to string
    r['estudios'] = p.Escolaridad #Map from id to string
    r['nacimiento'] = p.FechaNac #Map from id to string
    r['sangre'] = p.Sangre #Map from id to string

# Classify intervention

In [ ]:
for c, v in interventionToCase.items():
    interventionID = getProcFromXMLString(procedimientos.loc[c].XmlDescripcion)
    break

# New born information and birth

- New born anthropometrics
- Birth details (quirurgic)
- Life  /dead, and also of the mum
- Whether there was a single one, or many
- C- section

In [58]:
classificationProcedures = {'H0165': 'o', 'H0193': 'o', 'H2120': 'o', 'H2379': 'o', 'H2383': 'o', 'H2386': 'o', 'H2407': 'o', 'H2415': 'o', 'H2595': 'o', 'H2684': 'o', 'H2849': 'o', 'H2852': 'o', 'H2880': 'o', 'H2882': 'o', 'H2884': 'o', 'H2892': 'o', 'H2901': 'o', 'H2904': 'o', 'H2910': 'o', 'H2916': 'o', 'H2959': 'o', 'H2963': 'o', 'H2974': 'a', 'H2975': 'a', 'H3038': 'o', 'H3065': 'o', 'H3066': 'o', 'H3078': 'o', 'H3089': 'p', 'H3092': 'p', 'H3094': 'p', 'H3099': 'a', 'H3100': 'a', 'H3108': 'o', 'H3109': 'o', 'H3111': 'o', 'H3114': 'o', 'H3118': 'o', 'H4421': 'o', 'H4494': 'o', 'H4496': 'o', 'HE020': 'o'}

In [59]:
def findDesgarros(text):
    text = text.replace(' de ', ' ').replace(' se ', ' ')
    
    removeWords = ['lateral', 'izquierda', 'derecha', 'superior', 'inferior', 'medial']
    for w in removeWords:
        text = text.replace(' %s ' % w, ' ')

    #Perdida de sangre
    bloodLost = re.findall('perdida estimada sangre(?::)? ([0-9]+) (?:cc|ml)', text)
    ver = '(?:eviden[a-z]*|observ[a-z]*|vis[a-z]*|encont[a-z]*|presen[a-z]*)'   #diferentes manaeras de escribir ver
    negative = ['(?:sin|no) (?:%s )?desgar' % ver]
    positive = 'desgar[a-z]* (?:[a-z]* |(:?pared )?vag[a-z]* )?(?:sangr[a-z]* |no sangrant[a-z]* )?grado (i|ii|ii|1|2|3)'
    positiveUnidentified = '(?:%s )?desgar' % ver

    if re.findall('(%s)' % '|'.join(negative), text) or ('desgarro' not in text): #and 'sin complicaciones' in text):
        desgarro = 'no'
    elif re.findall(positive, text):
        desgarro = 'yes', re.findall(positive, text)
    elif re.findall(positiveUnidentified, text):
        desgarro = 'yes-NoGrade' + text
    else:
        desgarro = text
    return bloodLost, desgarro

In [ ]:
wrongParse = 0
for c,v in interventionToCase.items():
    xmlDesc = procedimientos.loc[c].XmlDescripcion
    idProc = getProcFromXMLString(procedimientos.loc[c].XmlDescripcion)
    if classificationProcedures[idProc] == 'p':
        et = ET.fromstring(xmlDesc)
        r=  findDesgarros(remove_diacritics(cleanString(et.find('detalle/procedimientos/procedimiento/descripcion').text.lower())))
        wrongParse += r[1] not in ['yes', 'no', 'yes-NoGrade']
        if r[1]  not in ['yes', 'no', 'yes-NoGrade']:
            print (r[1])
        casos = registersByCase.get_group(v[0])[registersByCase.get_group(v[0]).Asunto.str.contains('ALTA') == True]
        if len(casos):
            alta += 1
        else:
            noAlta += 1
            
        #Search for egreso 
        break

In [ ]:
registersByCase.get_group(v[0])

In [ ]:
print(registros.loc[661905].Asunto)
prettyPrintXML(registros.loc[661905].RegistroXML)


In [ ]:
registersByCase.get_group(v[0])

In [ ]:
prettyPrintXML(casos.iloc[0].RegistroXML)

# Abortion

In [ ]:
getProcFromXMLString(procedimientos.loc[17640].XmlDescripcion)

In [ ]:
procedimientos.loc[17640]

# Leaving the hospital
Search when they left the hospital, and in which state

In [36]:
caseToProcedure = {v[0] : k for  k,v in interventionToCase.items()}


In [37]:
pacientes.loc[152609]

Identificacion           VEN25322789
Carnet                           NaN
Nom1Afil                    HILDARIS
Nom2Afil                 FRANCHESSKA
Ape1Afil                       ARIZA
                            ...     
ParentescoResponsable            NaN
DirResponsable                   NaN
Creencia                         NaN
NombreResponsable                NaN
EstadoEmbarazo                   NaN
Name: 152609, Length: 86, dtype: object

In [94]:
import collections

def getInfoDischarge(r):
    et =ET.fromstring(r.RegistroXML)
    txt = cleanString(remove_diacritics(pars.unescape(findInXML('DescripcionNota', et)))).lower()
    peso = re.findall('pes[a-z]*\s+(%s)' % floatParse, txt)
    fecha = r.FechaAsignacionRegistro
    return
    #print(peso, fecha)
    if 'alta voluntaria' in txt:
        print('alta voluntaria')
    if 'cuidados intermedios' in txt:
        print('cuidados intermedios')
    elif 'cuidados basicos' in txt:
        print('cuidados basicos')
    elif 'alojamiento conjunto'in txt:
        print('alojamiento conjunto')
    elif 'alta medica'  in txt or 'alta hospitalaria' in txt:
        print('alta medica')
    else:
        print(txt)
    
def getPatient(c, registros):
    """
    Gets whether the record is of the mother or the newborn
    
    TODO: double check
    """
    asunto = remove_diacritics(str(c.Asunto)).lower()
    if 'neonato' in asunto or 'nacido' in asunto: 
        return  'neonato'
    elif 'Registro del recién nacido' == registros.loc[int(c.Padre)].Asunto:
        return  'neonato'
    else:
        return 'madre'
    
def isDischarge(c, registros):
    """
    Returns if there has been a discharge, and if it was of the mother or childs.
    """
    asunto = remove_diacritics(str(c.Asunto)).lower()
    posibilities = ['alta', 'salida', 'egreso', 'remision', 'traslado']
    if not any([ p in asunto for p in posibilities]):
        return False, _
    else:
        getInfoDischarge(c)
    
        if 'neonato' in asunto: 
            return True, 'neonato'
        elif 'Registro del recién nacido' == registros.loc[int(c.Padre)].Asunto:
            return True, 'neonato'
        else:
            return True, 'madre'

In [63]:
"""
Assumes that the procedure is 

TODO:
- twins
"""


discharges = collections.defaultdict(list)
for c, proc in tqdm.tqdm_notebook(caseToProcedure.items()):
    idProc = getProcFromXMLString(procedimientos.loc[proc].XmlDescripcion)
    if classificationProcedures[idProc] != 'p':
        continue
    r =  registersByCase.get_group(c)
    motherDischarge = False
    newBornDischarge = 0
    for id, v in r.iterrows():
        try:
            discharge, patient = isDischarge(v, registros)
            if discharge:
                discharges[c].append(v)
                if patient == 'mother':
                    motherDischarge = True
                else:
                    newBornDischarge += 1
        except Exception as e:
            print(e)
            print('error')
            pass
    # If discharge was not found, then get the last evolution, for both the mother and the newborn
    if motherDischarge == False:
        r[r.Asunto.map(lambda s: str(s).lower().contains('evol'))].sort_values(by=['FechaAsignacionRegistro'])
    
    if not c in discharges:
        print(c)

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  """


AD243427
AD246417
AD264263
AD266191
AD279679
AD287970
AD286278
AD293417
AD296017
AD296591
AD305327
AD312439
AD313110
AD314695
AD314928
AD315154
AD317068
AD320174
AD323071
AD325650
AD330031
AD330351
AD332050
AD332880
AD333190
AD333341
AD333598
AD333737
AD334063
AD333186
AD334548
AD334669
AD334550
AD333443
AD334207
AD335236
AD335431
AD335640
AD335893
AD336430
AD336261
AD336551
AD336650
AD336834
AD337150
AD337114
AD337212
AD334856
AD337275
AD337338
AD337359
AD337426
AD337451
AD337489
AD337639
AD338284
AD338298
AD338894
AD339069
AD339187
AD339266
AD339621
AD341088
AD341141
AD341131
AD341854
AD342043
AD342102
AD343683
AD342262
AD344271
AD344247
AD345047
AD345550
cannot convert float NaN to integer
error
AD346191
AD346830
AD348154
AD348228
AD348929
AD349023
AD349110
AD349249
AD348425
AD349627
AD349628
AD349765
AD350216
AD350521
AD350603
AD350689
AD350686
AD350798
AD350877
AD351069
AD351631
AD351023
AD353559
AD353708
AD353932
1302175
error
AD354123
AD353900
AD354259
1303855
error
AD353684
AD3

In [97]:

findInXML('FechaEgreso', et = ET.fromstring(registros.loc[1350618, 'RegistroXML']))

'13/12/2019 00:12'

In [96]:
prettyPrintXML(registros.loc[1350618, 'RegistroXML'])

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  from ipykernel import kernelapp as app


<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Ingreso de Urgencias" Caso="AD364505" CentroA="01" CodigoDiagnosticoRelacionado1="O623" CodigoDiagnosticoRelacionado2="O800" CodigoDiagnosticoRelacionado3="R102" Diagnostico="Z356" IdPaciente="217348" Modulo="01" Padre="AD364505" PlanAdm="20190409" Prestador="9068204" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- PERSONALES:
  7-&amp;gt; 1-&amp;gt; QX. NO
HOSPITALIZADA. NO.
ALERGICA. NO
FARMACOLOGICOS: NINGUNA 
HABITOS.NINGUNO
MENARQUIA: 12 A&#209;OS 
VIDA MARITAL: 15 A&#209;OS  
VIDA OBSTETRICA. 16 A&#209;OS
G1 P0 C0 A0 
CICLOS: 3X30 REGULARES
FUM: 10/03/19: EMB. 39.4 SEMANAS POR FUM
CONTROLES PRENATALES: 1

PARACLINICOS:
08/08/19 VIH: NEGATIVO
08/08/19 HEMOGRAMA: HB 10.0 HTO 33 LEU 9.300 LINF 48 

ECOGRAFIA:
28/05/19 EMBARAZO DE 10 SEMANAS Y 5 DIAS PARA HOY 39 SEMANAS
24/08/19 EMBARAZO DE 23 SEMA

In [95]:
registros[registros.Caso == 'AD364505']

,NumeroHistoria,Caso,CodigoRegistro,Asunto,FechaAsignacionRegistro,Raiz,Padre,Usuario,Prestador,AdmPlan,...,AltaMedica,IdRegistroPreanestesico,IdRegistroIntraOperatorio,IdRegistroPostOperatorio,Modulo,ProgramaPyP,FechaInicioAtencion,UnidadFuncional,ConfirmacionGuardado,Sincronizado
Id,,,,,,,,,,,,,,,,,,,,,
1350159,217348,AD364505,7,Ingreso de Urgencias,2019-12-12 11:21:21.683,NaN,NaN,GQUINTANA,3800965,20190409,...,False,NaN,NaN,NaN,3.0,NaN,2019-12-12 10:23:00,NaN,False,NaN
1350276,217348,AD364505,94,Notas de Ingreso a Piso,2019-12-12 18:06:45.147,NaN,NaN,KQUESADA,8865876,20190409,...,False,NaN,NaN,NaN,3.0,NaN,2019-12-12 18:02:00,NaN,False,NaN
1350296,217348,AD364505,145,NaN,2019-12-12 19:00:30.420,NaN,NaN,MRODRIGUEZ,33355330,20190409,...,False,NaN,NaN,NaN,3.0,NaN,NaT,NaN,False,NaN
1350297,217348,AD364505,165,Registro del recién nacido,2019-12-12 19:01:07.487,NaN,Padre,sa,1128060144,20190409,...,False,NaN,NaN,NaN,3.0,NaN,NaT,NaN,False,NaN
1350326,217348,AD364505,94,Notas de Ingreso a Piso,2019-12-12 21:37:46.180,NaN,NaN,RTRESPALACIOS,1047403836,20190409,...,False,NaN,NaN,NaN,1.0,NaN,2019-12-12 21:34:00,NaN,False,NaN
1350650,217348,AD364505,166,EVOLUCION MA&#209;ANA,2019-12-13 13:12:39.000,165.0,1350297,HARGOZ7,73086292,20190409,...,False,NaN,NaN,NaN,1.0,NaN,2019-12-13 13:05:00,NaN,False,NaN
1350368,217348,AD364505,106,Evoluci&#243;n PUERPERIO,2019-12-13 00:29:37.790,94.0,1350326,RTRESPALACIOS,1047403836,20190409,...,False,NaN,NaN,NaN,1.0,NaN,2019-12-13 00:12:00,NaN,False,NaN
1350618,217348,AD364505,450,Epicrisis de la Historia Ingreso de Urgencias,2019-12-13 12:09:20.333,1.0,AD364505,JRAMIREZ,9068204,20190409,...,False,NaN,NaN,NaN,1.0,NaN,NaT,NaN,False,NaN
1350624,217348,AD364505,46,Registro de Incapacidad,2019-12-13 12:16:41.560,NaN,NaN,JRAMIREZ,9068204,20190409,...,False,NaN,NaN,NaN,1.0,NaN,NaT,NaN,False,NaN


In [ ]:
for c,v in interventionToCase.items():
    xmlDesc = procedimientos.loc[c].XmlDescripcion
    idProc = getProcFromXMLString(procedimientos.loc[c].XmlDescripcion)
    if classificationProcedures[idProc] == 'p':
        print(v[0])
        r = registersByCase.get_group(v[0])
        [registersByCase.get_group(v[0]).Asunto.map(lambda s: remove_diacritics(str(s)).lower()).str.contains('salida recien')==True]
        print(registersByCase.get_group(v[0]))
        break

In [ ]:
noAlta = 0
for c, r in registersByCase.groups.items():
    if c in caseToProcedure and \
        classificationProcedures[getProcFromXMLString(procedimientos.loc[caseToProcedure[c], 'XmlDescripcion'])] == 'p':
        
        caseToProcedure[c]
        r = registersByCase.get_group(c)[registersByCase.get_group(c).Asunto.str.contains('ALTA')==True]
        r2 = registersByCase.get_group(c)[registersByCase.get_group(c).Asunto.str.contains('REMISION A CUIDADOS BASICOS')==True]
        r3 = registersByCase.get_group(c)[registersByCase.get_group(c).Asunto.str.contains('CUIDADOS INTERMEDIOS')==True]
        
        if len(r) == 0 and len(r2) == 0 and len(r3) == 0:
            #Try fecha egreso
            for c, k in registersByCase.get_group(c).iterrows():
                egreso = findInXML('FechaEgreso', ET.fromstring(k.RegistroXML))
                if egreso:
                    print('Found', egreso, findInXML('DestinoPaciente', ET.fromstring(k.RegistroXML)), findInXML('EstadoSalida', ET.fromstring(k.RegistroXML)))
                    print(c, k.Asunto)
                    break
            else:
                noAlta += 1
            

In [ ]:
noAlta

In [ ]:
prettyPrintXML(registros.loc[886499].RegistroXML)

In [ ]:
c

# Detect other factors